There seems to be a data leakabe problem in the dataset : <br>
https://towardsdatascience.com/distracted-driver-detection-using-deep-learning-e893715e02a4

Going to split on subject instead of randomly. The problem is that the current files do not keep track of the subject

Starting this notebook so that writing the hd5 files are all in one place


In [1]:
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
#['c'+str(x) for x in range(10)]

class_def = {'c0': 'safe driving',
'c1': 'texting - right',
'c2': 'talking on the phone - right',
'c3': 'texting - left',
'c4': 'talking on the phone - left',
'c5': 'operating the radio',
'c6': 'drinking',
'c7': 'reaching behind',
'c8': 'hair and makeup',
'c9': 'talking to passenger'}

In [2]:
import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random
#Create and load dataset
import h5py

In [3]:
# directories and filepaths
data_dir = Path(os.path.join(os.getcwd(),'data'))
base_dir = Path(os.path.join(data_dir, "state-farm-distracted-driver-detection"))
img_folder = Path(os.path.join(base_dir, 'imgs'))
train_imgs = Path(os.path.join(img_folder, 'train'))
test_imgs = Path(os.path.join(img_folder, 'test'))
# load the image lists
df = pd.read_csv(os.path.join(base_dir, 'driver_imgs_list.csv'))
sample_sub = pd.read_csv(os.path.join(base_dir, 'sample_submission.csv'))

In [4]:
# hdf5 filename 

# CHANGED FILENAMES TO INCLUDE GRAY OR COLOR 
#(THIS WILL BE A DIFF WITH THE LEAKAGE DATASETS THAT DIDNT HAVE DRIVER INFO)
downscale = 5
gray_scale=True
if gray_scale:
    hdf5_train = f"StateFarm_Train_Gray_{downscale}X.h5"
else:
    hdf5_train = f"StateFarm_Train_Color_{downscale}X.h5"

if gray_scale:    
    hdf5_test = f"StateFarm_Test_Gray_{downscale}X.h5"
else:
    hdf5_test = f"StateFarm_Test_Color_{downscale}X.h5"
#"StateFarm_Train_2X.h5" # scaled down by 2
#"StateFarm_Train_5X.h5" # scaled down by 5 

#WRITE (REWRITE) IMAGE HD5 FILES? 
write_dataset = True
write_testset = False 

In [5]:
df.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [6]:
# Look at number of images by subject
#plt.hist(df.subject.value_counts());
df.subject.value_counts()

# There are a few subjects that have a lot more instances than the others 

p021    1237
p022    1233
p024    1226
p026    1196
p016    1078
p066    1034
p049    1011
p051     920
p014     876
p015     875
p035     848
p047     835
p081     823
p012     823
p064     820
p075     814
p061     809
p056     794
p050     790
p052     740
p002     725
p045     724
p039     651
p041     605
p042     591
p072     346
Name: subject, dtype: int64

In [7]:
# conversion from string to array
str2int = lambda istr: int(re.findall('\d+', istr)[0])

df['driver'] =  df.subject.apply(str2int) # SID FOR SUBJECT ID
df['class'] = df.classname.apply(str2int) # 
df.head()

,subject,classname,img,driver,class
0,p002,c0,img_44733.jpg,2,0
1,p002,c0,img_72999.jpg,2,0
2,p002,c0,img_25094.jpg,2,0
3,p002,c0,img_69092.jpg,2,0
4,p002,c0,img_92629.jpg,2,0


In [8]:
# TODO : REMOVE WRITING FILES FROM SVM NOTEBOOK AND MAKE SVM NOTEBOOK ONLY HAVE PCA AND SVM
#
def rescale_image(filepath, resize_scale=2, gray_scale=True):
  '''
  Loads image, converts to grayscale, downsamples by resize_scale and returns a 
  To keep color image, set gray_scale = True 
  To keep the original size of the image, set resize_scale= 1
  '''
  im = Image.open(filepath)
  
  if gray_scale:
    im = im.convert('L')
    
  if resize_scale > 1:
    resize_dims = tuple([int(x/resize_scale) for x in (im.size)])
    im = im.resize(resize_dims)
    
  return np.expand_dims(np.array(im), axis=0)


In [9]:
if write_dataset: 
    # READ AND CONCATENATE TRAINING DATA
    # NOTE: USING NP ARRAYS IS MUCH SLOWER THAN DOING A LIST
    # For training images data set (could probably parallelize this but...)
    classnames = df['classname'].values
    c = df['class'].values
    driver = df['driver'].values
    filenames = df['img'].values
    image_list = [] # make a list, then concatenate
    for i, file in enumerate(filenames):
        if i%1000  == 0:    
            print(f"adding {file} to list, i={i}") 
        path_to_file = Path(os.path.join(train_imgs, classnames[i], file))        
        image_list.append(rescale_image(path_to_file, resize_scale = downscale, gray_scale=gray_scale)) 
    # concatenate images into singe nd array  
    images = np.concatenate(image_list, axis= 0)
    #writehdf5(data_dir, hdf5_train, images, labels)

adding img_44733.jpg to list, i=0
adding img_98718.jpg to list, i=1000
adding img_29411.jpg to list, i=2000
adding img_69390.jpg to list, i=3000
adding img_39358.jpg to list, i=4000
adding img_93192.jpg to list, i=5000
adding img_29731.jpg to list, i=6000
adding img_23637.jpg to list, i=7000
adding img_5799.jpg to list, i=8000
adding img_36717.jpg to list, i=9000
adding img_30792.jpg to list, i=10000
adding img_55189.jpg to list, i=11000
adding img_99251.jpg to list, i=12000
adding img_47864.jpg to list, i=13000
adding img_63132.jpg to list, i=14000
adding img_52243.jpg to list, i=15000
adding img_25404.jpg to list, i=16000
adding img_99194.jpg to list, i=17000
adding img_43583.jpg to list, i=18000
adding img_17664.jpg to list, i=19000
adding img_54984.jpg to list, i=20000
adding img_59295.jpg to list, i=21000
adding img_63401.jpg to list, i=22000


In [10]:
if write_dataset: 
    print('Writing HDF5 file')
    # Create a new HDF5 file
    file = h5py.File(
        data_dir / hdf5_train, "w"
    )

    # Create a dataset in the file
    image_set = file.create_dataset(
        "images",
        np.shape(images),
        h5py.h5t.STD_U8BE,
        data=images,
    )
    classnames_set = file.create_dataset(
        "c",
        np.shape(c),
        h5py.h5t.STD_U8BE,
        data=c,
    )    
    
    driver_set = file.create_dataset(
        "driver",
        np.shape(driver),
        h5py.h5t.STD_U8BE,
        data=driver,
    ) 
    
    file_set = file.create_dataset("fname", data=np.array(filenames, dtype='S'))
    
    file.close()      

Writing HDF5 file


In [11]:
# We want to split on the person id 
# However, not sure how the classes are represented by driver
drivers = np.unique(df['driver'].values)
ndrivers = len(drivers)

for i in range(ndrivers):
    print(df[df['driver']==drivers[i]].groupby(['driver', 'class']).count())
    


              subject  classname  img
driver class                         
2      0           76         76   76
       1           74         74   74
       2           86         86   86
       3           79         79   79
       4           84         84   84
       5           76         76   76
       6           83         83   83
       7           72         72   72
       8           44         44   44
       9           51         51   51
              subject  classname  img
driver class                         
12     0           84         84   84
       1           95         95   95
       2           91         91   91
       3           89         89   89
       4           97         97   97
       5           96         96   96
       6           75         75   75
       7           72         72   72
       8           62         62   62
       9           62         62   62
              subject  classname  img
driver class                         
14     0    

              subject  classname  img
driver class                         
61     0           84         84   84
       1           81         81   81
       2           81         81   81
       3           83         83   83
       4           79         79   79
       5           81         81   81
       6           80         80   80
       7           79         79   79
       8           81         81   81
       9           80         80   80
              subject  classname  img
driver class                         
64     0           83         83   83
       1           81         81   81
       2           83         83   83
       3           84         84   84
       4           86         86   86
       5           85         85   85
       6           82         82   82
       7           79         79   79
       8           81         81   81
       9           76         76   76
              subject  classname  img
driver class                         
66     0    

In [12]:
# HOW TO SPLIT DATASETS BY DRIVER: 

# All the drivers have data in all the classes. So that's good. 
# Still, because 50 and 51 seem to be really safe drivers, 
# going to split on a shuffle of the drivers

# wanting to do roughly 80/20 split but keeping the drivers separate
# get npts per driver
rng = np.random.default_rng(seed=42) 
nPerDriver = df.groupby('driver').count().values
nPerDriver = nPerDriver[:,0]
driver_idx  = rng.permutation(ndrivers)
# shuffle the drivers
drivers = drivers[driver_idx]
nPerDriver = nPerDriver[driver_idx]

In [13]:
train_log = np.cumsum(nPerDriver) < (22424*.8) # really close to 20/80 plit (splits at 17891, 80% is 17939)
test_log = np.logical_not(train_log)
train_drivers = drivers[train_log]
test_drivers = drivers[test_log]

In [14]:
# make sure there's no intersection
np.intersect1d(train_drivers, test_drivers).size ==0

True

In [15]:
 np.cumsum(nPerDriver)

array([ 1226,  2237,  3057,  3866,  4786,  6023,  6763,  7586,  8237,
        9085, 10318, 11108, 11983, 12708, 13742, 14536, 15127, 15732,
       16567, 17381, 18257, 19335, 19681, 20504, 21228, 22424],
      dtype=int64)

In [16]:

str2int = lambda istr: int(re.findall('\d+', istr)[0])

def split_on_driver(df, train_split=.8,  seed=0):
    # splits dataframe based on drivers
    # take in dataframe (original), train_split proportion,  random seed
    # returns new dataframe, train_drivers,  test_drivers
    
    # if dataframe doesn't already have driver, addit 
    if 'driver' not in df.columns:
        df['driver'] =  df.subject.apply(str2int) # SID FOR SUBJECT ID
        df['class'] = df.classname.apply(str2int) # 
   
    # random number generator
    rng = np.random.default_rng(seed=seed) 
    
    # unique drivers and number of drivers
    drivers = np.unique(df['driver'].values)
    ndrivers = len(drivers)
    
    # number of data pts per driver 
    nPerDriver = df.groupby('driver').count().values
    nPerDriver = nPerDriver[:,0]
    
    # shuffle drivers (get shuffle indices)
    shuff_idx  = rng.permutation(ndrivers)
    
    # shuffle the drivers and the nPerDriver according to shuffle indices
    drivers = drivers[shuff_idx]
    nPerDriver = nPerDriver[shuff_idx]
    
    # separate drivers according to train_split
    train_log = np.cumsum(nPerDriver) < (22424*train_split) # really close to 20/80 plit (e.g. splits at 17891, 80% is 17939)
    test_log = np.logical_not(train_log)
    train_drivers = drivers[train_log]
    test_drivers = drivers[test_log]
    
    return (df, train_drivers, test_drivers, nPerDriver)

In [17]:
df2, train_drivers, test_drivers, nPerDriver = split_on_driver(df, .8, 0)

In [18]:
train_drivers

array([56, 16, 39, 41, 81, 14, 75, 22, 50, 72, 15, 64, 26,  2, 61, 42, 52,
       45, 24, 21, 51], dtype=int64)

In [19]:
np.cumsum(nPerDriver[0:train_drivers.size+1])

array([  794,  1872,  2523,  3128,  3951,  4827,  5641,  6874,  7664,
        8010,  8885,  9705, 10901, 11626, 12435, 13026, 13766, 14490,
       15716, 16953, 17873, 18708], dtype=int64)